In [ ]:
import json
from typing import AsyncGenerator, List, Sequence
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage




async def goal_decomposition_result(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."

async def run_goal_decomposition_agent(goal: str) -> None:
    # Create a countdown agent.
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o-mini",
        api_key=open("api_key").read(),
    )
    goal_decomposition_agent = AssistantAgent(
        name="goal_decomposition_agent", 
        model_client=model_client,
        system_message="""You are a text analytics task planner. Users have collected a dataset, and they need help with text analytics tasks.  Users will describe a goal to you, and you need to help them decompose the goal into subtasks, and then provide a plan to complete each of the subtask.
        Reply with this JSON format:
{
    "steps": [
        {
            "label": (string)
            "description": (string)
            "explanation": (string, explain why this step is needed)
        },
        {
            "label": (string)
            "description": (string)
            "explanation": (string, explain why this step is needed)
        },
        ...
    ]
}  """)
    response = await goal_decomposition_agent.on_messages(
        [TextMessage(content=goal, source="user")],
        cancellation_token=CancellationToken(),
    )
    return json.loads(response.chat_message.content)
    print(response.chat_message)

    # Run the agent with a given task and stream the response.

# Use asyncio.run(run_countdown_agent()) when running in a script.
goal = "I need to construct a knowledge graph from a collection of documents from wikipedia."
response = await run_goal_decomposition_agent(goal=goal)

In [16]:
import json
json.loads(response.content)

{'steps': [{'label': 'Document Collection',
   'description': 'Compile the selected Wikipedia documents that will serve as the data source for the knowledge graph.',
   'explanation': 'This step is essential to ensure that you have all relevant content gathered in one place to work with. A defined set of documents will help focus the analysis and extraction process.'},
  {'label': 'Text Preprocessing',
   'description': 'Clean and normalize the text from the collected documents to remove noise and facilitate entity extraction.',
   'explanation': 'Preprocessing ensures that the data is in a usable format, including lowercasing, removing punctuation, and filtering out stop words. Clean text improves the accuracy of subsequent steps like entity recognition.'},
  {'label': 'Entity Recognition',
   'description': 'Utilize Named Entity Recognition (NER) tools or libraries to identify and extract entities (e.g., persons, organizations, locations) from the cleaned text.',
   'explanation': 'I